<a href="https://colab.research.google.com/github/winda04/coursera/blob/main/capstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Load the Dataset From Kaggle

In [1]:
! pip install opendatasets pandas numpy matplotlib seaborn folium --q

In [2]:
import opendatasets as od
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import lightgbm as lgb
from sklearn.model_selection import KFold
import datetime
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error

2. Download data From Kaggle

In [10]:
od.download('https://www.kaggle.com/competitions/ashrae-energy-prediction/data')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: windaastarisiagian
Your Kaggle Key: ··········


100%|██████████| 379M/379M [00:05<00:00, 71.1MB/s]



Extracting archive ./ashrae-energy-prediction/ashrae-energy-prediction.zip to ./ashrae-energy-prediction


In [11]:
data_dir = './ashrae-energy-prediction'

#view dataset
# List of files with size
!ls -lh {data_dir}

In [13]:
# Training set
!head {data_dir}/train.csv

building_id,meter,timestamp,meter_reading
0,0,2016-01-01 00:00:00,0.0
1,0,2016-01-01 00:00:00,0.0
2,0,2016-01-01 00:00:00,0.0
3,0,2016-01-01 00:00:00,0.0
4,0,2016-01-01 00:00:00,0.0
5,0,2016-01-01 00:00:00,0.0
6,0,2016-01-01 00:00:00,0.0
7,0,2016-01-01 00:00:00,0.0
8,0,2016-01-01 00:00:00,0.0


In [14]:
# building set
!head {data_dir}/building_metadata.csv

site_id,building_id,primary_use,square_feet,year_built,floor_count
0,0,Education,7432,2008,
0,1,Education,2720,2004,
0,2,Education,5376,1991,
0,3,Education,23685,2002,
0,4,Education,116607,1975,
0,5,Education,8000,2000,
0,6,Lodging/residential,27926,1981,
0,7,Education,121074,1989,
0,8,Education,60809,2003,


In [15]:
# Weather Training set
!head {data_dir}/weather_train.csv

site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,2016-01-01 00:00:00,25.0,6.0,20.0,,1019.7,0.0,0.0
0,2016-01-01 01:00:00,24.4,,21.1,-1.0,1020.2,70.0,1.5
0,2016-01-01 02:00:00,22.8,2.0,21.1,0.0,1020.2,0.0,0.0
0,2016-01-01 03:00:00,21.1,2.0,20.6,0.0,1020.1,0.0,0.0
0,2016-01-01 04:00:00,20.0,2.0,20.0,-1.0,1020.0,250.0,2.6
0,2016-01-01 05:00:00,19.4,,19.4,0.0,,0.0,0.0
0,2016-01-01 06:00:00,21.1,6.0,21.1,-1.0,1019.4,0.0,0.0
0,2016-01-01 07:00:00,21.1,,21.1,0.0,1018.8,210.0,1.5
0,2016-01-01 08:00:00,20.6,,20.0,0.0,1018.1,0.0,0.0


In [16]:
# Test set
!head {data_dir}/test.csv

row_id,building_id,meter,timestamp
0,0,0,2017-01-01 00:00:00
1,1,0,2017-01-01 00:00:00
2,2,0,2017-01-01 00:00:00
3,3,0,2017-01-01 00:00:00
4,4,0,2017-01-01 00:00:00
5,5,0,2017-01-01 00:00:00
6,6,0,2017-01-01 00:00:00
7,7,0,2017-01-01 00:00:00
8,7,1,2017-01-01 00:00:00


In [17]:
# Weather Test set
!head {data_dir}/weather_test.csv

site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,2017-01-01 00:00:00,17.8,4.0,11.7,,1021.4,100.0,3.6
0,2017-01-01 01:00:00,17.8,2.0,12.8,0.0,1022.0,130.0,3.1
0,2017-01-01 02:00:00,16.1,0.0,12.8,0.0,1021.9,140.0,3.1
0,2017-01-01 03:00:00,17.2,0.0,13.3,0.0,1022.2,140.0,3.1
0,2017-01-01 04:00:00,16.7,2.0,13.3,0.0,1022.3,130.0,2.6
0,2017-01-01 05:00:00,15.6,2.0,12.8,0.0,1022.1,130.0,2.1
0,2017-01-01 06:00:00,15.0,0.0,12.8,0.0,1021.7,150.0,1.5
0,2017-01-01 07:00:00,15.0,2.0,13.3,0.0,1021.6,0.0,0.0
0,2017-01-01 08:00:00,13.3,0.0,12.2,0.0,1021.4,0.0,0.0


In [18]:
# Submission set
!head {data_dir}/sample_submission.csv

row_id,meter_reading
0,0
1,0
2,0
3,0
4,0
5,0
6,0
7,0
8,0


In [19]:
# No. of lines in training set
!wc -l {data_dir}/train.csv

20216101 ./ashrae-energy-prediction/train.csv


In [20]:
# No. of lines in weather training set
!wc -l {data_dir}/weather_train.csv

139774 ./ashrae-energy-prediction/weather_train.csv


In [21]:
# No. of lines in building set
!wc -l {data_dir}/building_metadata.csv

1450 ./ashrae-energy-prediction/building_metadata.csv


In [22]:
# No. of lines in test set
!wc -l {data_dir}/test.csv

41697601 ./ashrae-energy-prediction/test.csv


In [23]:
# No. of lines in weather test set
!wc -l {data_dir}/weather_test.csv

277244 ./ashrae-energy-prediction/weather_test.csv


In [24]:
# No. of lines in submission set
!wc -l {data_dir}/sample_submission.csv

41697601 ./ashrae-energy-prediction/sample_submission.csv


**LOADING TRAINING SET**

for optimizations we can do


1.   Parse pickup datetime while loading data
2.   Specify data types for other columns :
*   int16 for building_id
*   int8 for meter
*   float32 for meter_reading
3.   Work with just 1% sample of the data






In [25]:
%%time
import random
#sample_frac=0.05
#selected_cols = 'building_id,meter,timestamp,meter_reading'.split(',')
dtypes = {
    'building_id':'int16',
    'meter':'int8',
    'meter_reading':'float32'

}

def skip_row(row_idx):
    if row_idx == 0:
        return False
    return random.random() > sample_frac

random.seed(42)
train_df = pd.read_csv(data_dir+"/train.csv",
                 #usecols=selected_cols,
                 dtype=dtypes,
                 parse_dates=['timestamp'])
                 #skiprows=skip_row)

CPU times: user 11.9 s, sys: 1.15 s, total: 13.1 s
Wall time: 13.1 s


In [26]:
train_df.sample(5)

,building_id,meter,timestamp,meter_reading
14554556,512,0,2016-09-22 04:00:00,4.280000
3410281,962,1,2016-03-05 02:00:00,533.541016
14294644,1196,0,2016-09-17 13:00:00,111.037003
8824900,856,0,2016-06-11 22:00:00,1.333300
19680804,287,1,2016-12-22 13:00:00,1.160600


**Loading Building Data**

In [27]:
building_dtypes={
    'site_id':'int8',
    'building_id':'int16',
    'square_feet':'int32',
    'year_built':'float16',
    'floor_count':'float16'}

In [28]:
building_df=pd.read_csv('/content/ashrae-energy-prediction/building_metadata.csv',dtype=building_dtypes)

In [29]:
building_df.sample(5)

,site_id,building_id,primary_use,square_feet,year_built,floor_count
1352,15,1352,Office,42755,NaN,NaN
813,8,813,Public services,5630,NaN,1.0
1064,12,1064,Education,131212,NaN,NaN
553,3,553,Education,37800,NaN,NaN
686,5,686,Office,12206,1976.0,2.0


**Loading Weather Train/Test Dataset**

In [30]:
weather_dtypes={
    'site_id':'int8',
    'air_temperature':'float16',
    'cloud_coverage':'float16',
    'dew_temperature':'float16',
    'precip_depth_1_hr':'float16',
    'sea_level_pressure':'float16',
    'wind_direction':'float16',
    'wind_speed':'float16',
}

In [31]:
weather_train_df=pd.read_csv('/content/ashrae-energy-prediction/weather_train.csv',dtype=weather_dtypes,parse_dates=['timestamp'])

In [32]:
weather_train_df.sample(5)

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
20,0,2016-01-01 20:00:00,28.296875,NaN,18.906250,0.0,1017.0,210.0,4.101562
70732,8,2016-01-29 16:00:00,13.898438,4.0,3.900391,0.0,1019.0,300.0,6.199219
40317,4,2016-08-05 00:00:00,17.796875,NaN,12.203125,0.0,1013.5,280.0,7.699219
135279,15,2016-06-21 17:00:00,22.796875,2.0,6.699219,NaN,1012.0,300.0,6.199219
115298,13,2016-03-05 03:00:00,1.700195,NaN,0.000000,-1.0,1014.0,230.0,3.599609


In [33]:
weather_test_df=pd.read_csv('/content/ashrae-energy-prediction/weather_test.csv',dtype=weather_dtypes,parse_dates=['timestamp'])

In [34]:
weather_test_df.sample(5)

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
138663,7,2018-12-31 09:00:00,-6.500000,NaN,-8.203125,NaN,1023.5,70.0,2.599609
101472,5,2018-08-13 20:00:00,17.000000,0.0,13.000000,NaN,NaN,260.0,1.500000
226030,13,2017-02-12 15:00:00,-1.099609,0.0,-7.199219,0.0,1024.0,310.0,6.699219
252674,14,2018-02-27 22:00:00,12.796875,0.0,-5.000000,0.0,1024.0,250.0,2.599609
96266,5,2018-01-07 18:00:00,3.000000,0.0,-2.000000,NaN,NaN,60.0,10.796875


**Loading Test Set**

In [36]:
# chunk_size=10424401
# batch_no=0
# for chunk in pd.read_csv('/content/ashrae-energy-prediction/test.csv',dtype=dtypes,parse_dates=['timestamp'],chunksize=chunk_size):
#   batch_no+=1
#   if batch_no==1:
#     test1_df=chunk.merge(building_df,on='building_id',how='left')
#     test1_df=test1_df.merge(weather_test_df,how='left',left_on=['site_id','timestamp'],right_on=['site_id','timestamp'])
#   elif batch_no==2:
#     test2_df=chunk.merge(building_df,on='building_id',how='left')
#     test2_df=test2_df.merge(weather_test_df,how='left',left_on=['site_id','timestamp'],right_on=['site_id','timestamp'])
#   elif batch_no==3:
#     test3_df=chunk.merge(building_df,on='building_id',how='left')
#     test3_df=test3_df.merge(weather_test_df,how='left',left_on=['site_id','timestamp'],right_on=['site_id','timestamp'])
#   else:
#     test4_df=chunk.merge(building_df,on='building_id',how='left')
#     test4_df=test4_df.merge(weather_test_df,how='left',left_on=['site_id','timestamp'],right_on=['site_id','timestamp'])

test_df=pd.read_csv('/content/ashrae-energy-prediction/test.csv',dtype=dtypes,parse_dates=['timestamp'])

In [37]:
test_df.sample(5)

,row_id,building_id,meter,timestamp
27661426,27661426,1202,0,2017-05-05 04:00:00
22447553,22447553,987,0,2018-01-15 14:00:00
18147929,18147929,803,2,2018-03-04 04:00:00
33423284,33423284,1245,0,2017-07-03 17:00:00
32948290,32948290,1319,1,2017-04-25 23:00:00


In [41]:
git add test.py  # Untuk menambahkan satu file
git add .             # Untuk menambahkan semua perubahan

SyntaxError: ignored